In [481]:
import pandas as pd
import numpy as np
import itertools as it
import networkx as nx
# Documentacion de la libreria: http://networkx.readthedocs.io/en/networkx-1.11/

In [482]:
def filterData(df, isTraining, date):
    """
        Funcion que devuelve el conjunto de problemas que tienen status AC o PE
        Si isTraining es true, entonces la funcion sacara el training_set, si no, sacara el evaluation_set
        date es la fecha de particion
    """
    
    if isTraining:
        df = df[df['submissionDate'] < date]
        df = df.loc[df['status'].isin(['AC', 'PE'])]
    else:
        df = df[df['submissionDate'] >= date]
    
    

    return df

In [483]:
# MAIN
# ---------

# se guarda en la variable df (DataFrame) toda la base de datos
df = pd.read_csv('bbdd_orderbydate.csv')

# aqui quito los problemas que no existian despues de la fecha umbral
df = df[df['problem_id'] <= 511] 

# construyo el conjunto de entrenamiento
training_set = filterData(df, True, "2016-10-21 00:00:00")

print(training_set)

# obtengo los nodos del grafo, esta vez los nodos son los usuarios y no los problemas:
nodes = training_set.user_id.unique()

# creo un diccionario que va a tener a los usuarios como keys y los valores seran los
# problemas que ha hecho ese usuario
grouped = training_set.groupby('user_id')['problem_id'].apply(list)

# muestra el numero de usuarios
print(len(nodes))

# muestra la lista de problemas que ha hecho cada usuario
print(grouped)

       problem_id  user_id status       submissionDate
0              10        5     AC  2014-02-17 15:27:07
1               2        6     AC  2014-02-17 15:39:17
2               2        9     AC  2014-02-18 00:30:14
3              10        9     AC  2014-02-18 00:34:46
4               4        9     AC  2014-02-18 00:50:28
5               6        9     AC  2014-02-18 00:52:11
6              13        9     AC  2014-02-18 00:53:40
9              15        8     AC  2014-02-19 19:58:03
10              4        8     AC  2014-02-20 14:23:30
11             39       16     AC  2014-02-20 15:44:33
12             39       17     AC  2014-02-20 16:53:34
13             13       12     AC  2014-02-21 11:08:38
15             33       12     AC  2014-02-21 11:58:32
16             39       12     AC  2014-02-21 12:05:49
17             44       12     AC  2014-02-21 12:52:44
18             44       15     AC  2014-02-21 14:51:52
19             44       22     AC  2014-02-21 14:58:55
21        

In [484]:
print(training_set)

       problem_id  user_id status       submissionDate
0              10        5     AC  2014-02-17 15:27:07
1               2        6     AC  2014-02-17 15:39:17
2               2        9     AC  2014-02-18 00:30:14
3              10        9     AC  2014-02-18 00:34:46
4               4        9     AC  2014-02-18 00:50:28
5               6        9     AC  2014-02-18 00:52:11
6              13        9     AC  2014-02-18 00:53:40
9              15        8     AC  2014-02-19 19:58:03
10              4        8     AC  2014-02-20 14:23:30
11             39       16     AC  2014-02-20 15:44:33
12             39       17     AC  2014-02-20 16:53:34
13             13       12     AC  2014-02-21 11:08:38
15             33       12     AC  2014-02-21 11:58:32
16             39       12     AC  2014-02-21 12:05:49
17             44       12     AC  2014-02-21 12:52:44
18             44       15     AC  2014-02-21 14:51:52
19             44       22     AC  2014-02-21 14:58:55
21        

In [485]:
# OBTENCION DEL EVALUATION_SET
# -------

# ahota saco el evaluation_set
evaluation_set = filterData(df, False, "2016-10-21 00:00:00")

print(evaluation_set)

# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user_eval = evaluation_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users_eval = pd.DataFrame({'user_id':grouped_user_eval.index, 'list_problem_id':grouped_user_eval.values})

print(df_users_eval)

       problem_id  user_id status       submissionDate
16939         469      799     RF  2016-10-21 00:08:23
16940         325     3832     AC  2016-10-21 00:08:35
16942         469     3757     AC  2016-10-21 00:41:36
16943         469     3792    RTE  2016-10-21 01:04:08
16944         469      810     AC  2016-10-21 01:48:35
16945         438     3943     AC  2016-10-21 02:23:56
16946         469     3773     AC  2016-10-21 05:00:58
16947         346     3728     AC  2016-10-21 09:06:38
16948         469     3736     CE  2016-10-21 09:38:14
16949         438     3785     TL  2016-10-21 09:43:52
16950         469     3704     AC  2016-10-21 10:04:03
16951         469     2912     AC  2016-10-21 10:20:36
16952         213     3136     AC  2016-10-21 10:28:23
16953         213     3088     AC  2016-10-21 10:29:26
16954         469     3708    RTE  2016-10-21 10:35:15
16956         235     3136    RTE  2016-10-21 10:37:58
16957         183     3880     AC  2016-10-21 11:38:11
16958     

In [486]:
# In[3]:

def compareNodes(f_list, s_list):
    """
        Funcion que devuelve el numero de usuarios que han hecho ambos problemas
    """
    peso = len(np.intersect1d(f_list, s_list))
    
    return peso
    
def createLinks(prob_us_set, nodos):
    """
        Funcion que crea los enlaces del grafo a partir de la informacion contenida en el conjunto que se le
        pasa a la funcion
    """
    resultado = list() 
    
    # hago todas las posibles combinaciones de usuarios
    for fst, snd in it.combinations(nodos, 2):
        # obtengo el peso pasando la lista de problemas que ha hecho cada usuario
        peso = compareNodes(prob_us_set[fst], prob_us_set[snd])
        if peso >= 1:
            resultado.append((fst, snd, peso))
            
            
            
    return resultado


# In[4]:

def filterWeight(weightUmbral, linksToFilter):
    """
        Funcion que filtra los enlaces de un grafo, para que el peso sea mayor o igual al dado
    """
    
    result = [(x, y, z) for (x, y, z) in linksToFilter if z >= weightUmbral]
    
    return result
    


# In[5]:

def create_graph_nx(list_nodes, list_links):
    """
        Funcion que crea un grafo de tipo Graph de la libreria NetworkX
        Construccion del grafo: http://networkx.readthedocs.io/en/networkx-1.11/tutorial/tutorial.html#what-to-use-as-nodes-and-edges
    """
    grafo = nx.Graph() # creo la variable grafo

    # incluyo los nodos del grafo 
    grafo.add_nodes_from(list_nodes)

    # se incluyen las tuplas de enlaces con el peso del enlace
    # es una lista de la forma [(Nodo1, Nodo2, peso), ......]
    grafo.add_weighted_edges_from(list_links)

    return grafo


In [487]:
# creo los enlaces a partir de la informacion de los nodos
links = createLinks(grouped, nodes)
# ahora filtro el grafo para que los enlaces solo tengan el peso que quiero
linksFiltered = filterWeight(5, links)

print(len(linksFiltered))

# aqui creo el grafo 
graph = create_graph_nx(nodes, linksFiltered)

16805


In [488]:
def lenProblemsDone(row, set_filter):
    """
        Funcion auxiliar que calcula cuanto problemas ha hecho cada usuario en un conjunto: training o evaluation
    """
    # saco el dataframe que contendra solo una fila con la lista de problemas que ha hecho el usuario
    df_filter = set_filter[set_filter['user_id'] == row['user_id']]
    
    if df_filter.empty:
        # si esta vacio, entonces es que el usuario no ha hecho problemas en ese conjunto
        return 0
    else:
        # sino, devuelvo la longitud de la lista de problemas
        return len(df_filter['list_problem_id'].iloc[0]) 
    

In [489]:
# aqui voy a hacer el filtro de usuarios de forma que para hacer las recomendaciones solo tengamos en 
# cuenta aquellos usuarios que han hecho 5 o mas problemas tanto antes de la fecha limite como despues

# convierto la serie en un dataframe
df_users = pd.DataFrame({'user_id':grouped.index, 'list_problem_id':grouped.values})

# primero guardo la lista de usuarios
user_list = df.user_id.unique()

# la meto en un dataframe 
column_user_filter = {'user_id': user_list}
datraframe_user_filter = pd.DataFrame.from_dict(column_user_filter)


# ahora tengo que calcular para cada fila, el numero de problemas que han hecho en el training_set, evaluation_set
datraframe_user_filter['len_training'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users), axis=1)
datraframe_user_filter['len_evaluation'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users_eval), axis=1)
print(datraframe_user_filter)

      user_id  len_training  len_evaluation
0           5             4               1
1           6             2               0
2           9            11               3
3           8            47               0
4          16             1               0
5          17             2               2
6          12            12               0
7          18             5               8
8          15             3               1
9          22             1               0
10          7             7               0
11         27             0               0
12         28             1               0
13         29             1               0
14         32             2               0
15         35            41               1
16         33             9               0
17         39            11               0
18         31             9               0
19         40             1               0
20         44             8               0
21         48             0     

In [490]:
# ahora tengo que hacer el filtro en este dataframe, de forma que solo aparezcan las filas en las que len_training y 
# len_evaluation sea >=5
datraframe_user_filter = datraframe_user_filter[(datraframe_user_filter['len_training'] >= 5) & (datraframe_user_filter['len_evaluation'] >=5)]
print(datraframe_user_filter)

# aqui voy a guardar la lista de usuarios a los que voy a recomendar
user_list_to_recommend = sorted(datraframe_user_filter['user_id'].tolist())
print(user_list_to_recommend)
print(len(user_list_to_recommend))

      user_id  len_training  len_evaluation
7          18             5               8
29         60            18              14
118        25            22               7
261       130            80               6
299       414            49              61
316       443            53               8
317       448            43              21
483       689            33               7
619       912            13              11
633       935           127               9
1196     1711           128              16
1288     1893             7               6
1311     1952            19               9
1339     1955             9               5
1416     2038             5               5
1423     2096            12              13
1446     2051            11               8
1447     2025            12               8
1448     2120             7               8
1451     2041             9               7
1528     2257             6              13
1694     2576             5     

In [491]:
# ahora tengo que filtrar df_users para que solo contenga las filas en las que los usuarios
# pertenecen a la anterior lista

df_users_recommend = df_users[df_users['user_id'].isin(user_list_to_recommend)]
print(df_users_recommend)

                                        list_problem_id  user_id
11                            [119, 114, 316, 507, 506]       18
14    [258, 141, 309, 310, 390, 187, 510, 438, 13, 2...       25
39    [100, 62, 2, 57, 53, 23, 134, 15, 147, 233, 24...       60
90    [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130
241   [213, 2, 109, 114, 10, 436, 438, 437, 404, 239...      414
255   [39, 100, 247, 150, 183, 44, 471, 109, 307, 30...      443
257   [255, 257, 314, 311, 315, 310, 282, 243, 275, ...      448
392   [171, 272, 282, 209, 373, 155, 2, 39, 134, 471...      689
512   [241, 269, 2, 340, 65, 379, 316, 275, 309, 259...      912
523   [65, 241, 239, 105, 231, 269, 275, 331, 257, 1...      935
933   [507, 33, 83, 217, 312, 256, 49, 4, 503, 254, ...     1711
997                      [39, 49, 465, 159, 134, 13, 2]     1893
1021  [325, 441, 237, 183, 438, 510, 226, 4, 327, 21...     1952
1024      [325, 327, 441, 468, 471, 503, 226, 109, 446]     1955
1066  [155, 505, 49, 162,

In [492]:

# en df_new tengo los usuarios a los que tengo que hacer recomendaciones

# primero guardo la lista de usuarios
user_list_recomend = df_users_recommend.user_id.unique()

# creo el nuevo dataframe con los resultados 
column_user_recomend = {'user_id': user_list_to_recommend}
dataframe_user_recomend = pd.DataFrame.from_dict(column_user_recomend)

print(dataframe_user_recomend)

    user_id
0        18
1        25
2        60
3       130
4       414
5       443
6       448
7       689
8       912
9       935
10     1711
11     1893
12     1952
13     1955
14     2025
15     2027
16     2038
17     2041
18     2051
19     2096
20     2106
21     2120
22     2257
23     2355
24     2373
25     2576
26     2626
27     2906
28     2966
29     3118
..      ...
35     3708
36     3710
37     3712
38     3714
39     3722
40     3724
41     3727
42     3731
43     3735
44     3749
45     3759
46     3779
47     3784
48     3795
49     3842
50     3846
51     3847
52     3855
53     3859
54     3873
55     3880
56     3890
57     3931
58     3937
59     3938
60     3941
61     3946
62     3961
63     3963
64     3969

[65 rows x 1 columns]


In [493]:
def apply_aa(row, graph):
    """
        Funcion que devuelve para cada par de nodos, el sumatorio de 1/log(N(z)), siendo N(z) el grado del nodo z para todo z 
        perteneciente al conjunto de nodos en comun de ese par de nodos
    """

    # obtengo un iterador de un solo elemento que tiene en la tercera posicion el valor de AA para el par de nodos
    value = nx.adamic_adar_index(graph, [(row['one'], row['two'])])
    
    value_aa = -1
    for u, v, p in value:
        # print(p)
        # itero el iterador, guardando el valor de adar adamic
        value_aa = p

    return value_aa

def create_aa_data(graph, nodes):

    # Ahora voy a construir un DataFrame que tenga dos columnas con todas las posibles combinaciones de problemas, y otra 
    # columna con el valor de aa para ese par de problemas
    fst_column = list()
    snd_column = list()
    for fst, snd in it.combinations(nodes, 2):
        fst_column.append(fst)
        snd_column.append(snd)

    d = {'one' : fst_column,
        'two' : snd_column}
    dataFrame_aa = pd.DataFrame(d)


    # Aplico la funcion a cada fila
    dataFrame_aa['aa'] = dataFrame_aa.apply (lambda row: apply_aa(row, graph), axis=1)


    return dataFrame_aa

In [494]:
aa_df = create_aa_data(graph, nodes)
print(aa_df)

          one   two   aa
0           5     6  0.0
1           5     9  0.0
2           5     8  0.0
3           5    16  0.0
4           5    17  0.0
5           5    12  0.0
6           5    15  0.0
7           5    22  0.0
8           5     7  0.0
9           5    28  0.0
10          5    29  0.0
11          5    32  0.0
12          5    35  0.0
13          5    33  0.0
14          5    31  0.0
15          5    40  0.0
16          5    49  0.0
17          5    42  0.0
18          5    46  0.0
19          5    24  0.0
20          5    53  0.0
21          5    60  0.0
22          5    59  0.0
23          5    66  0.0
24          5    68  0.0
25          5    71  0.0
26          5    69  0.0
27          5    70  0.0
28          5    65  0.0
29          5    73  0.0
...       ...   ...  ...
2012991  3699  2046  0.0
2012992  3699  4018  0.0
2012993  3997  4011  0.0
2012994  3997  3951  0.0
2012995  3997  3949  0.0
2012996  3997  3952  0.0
2012997  3997  3982  0.0
2012998  3997  2046  0.0


In [495]:
def getSimilarUsers(row, aa_df):
    """
        Funcion que devuelve la lista de los usuarios de ese usuario que tienen vecinos en comun
    """
    # print(row['user_id'])
    
    # obtengo dos df con los usuarios que tienen usuarios vecinos con el usuario de la fila 
    column_result_one_tmp = aa_df[aa_df['one'] == row['user_id']]
    column_result_one = column_result_one_tmp[column_result_one_tmp['aa'] > 0]
    column_result_two_tmp = aa_df[aa_df['two'] == row['user_id']]
    column_result_two = column_result_two_tmp[column_result_two_tmp['aa'] > 0]
    
    # saco las listas de usuarios con usuarios comunes
    list_one = list(column_result_one['two'])
    list_two = list(column_result_two['one'])
    
    # la concateno sin tener en cuenta repeticiones, porque nunca va a haber
    list_aa = list_one + list_two
    
    # print(list_aa)
    
    if list_aa == []: # sino tiene vecinos en comun, pongo toda la lista de nodos
        list_aa = graph.nodes()
        list_aa.remove(row['user_id']) # y elimino el nodo que estoy mirando
    
    # hago el filtro de los k mejores
    return list_aa

In [496]:
dataframe_user_recomend['neighbors'] = dataframe_user_recomend.apply (lambda row: getSimilarUsers(row, aa_df), axis=1)

# aqui tengo la lista de usuarios con sus k usuarios similares
print(dataframe_user_recomend)

    user_id                                          neighbors
0        18  [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...
1        25  [418, 437, 440, 443, 448, 445, 302, 454, 130, ...
2        60  [67, 64, 84, 89, 91, 54, 95, 82, 112, 120, 124...
3       130  [461, 396, 483, 103, 488, 416, 509, 511, 527, ...
4       414  [176, 25, 418, 437, 440, 443, 448, 445, 302, 4...
5       443  [448, 445, 302, 454, 130, 461, 396, 483, 103, ...
6       448  [445, 302, 454, 130, 461, 396, 483, 103, 488, ...
7       689  [709, 713, 676, 720, 724, 758, 771, 712, 722, ...
8       912  [840, 919, 864, 930, 933, 813, 934, 935, 105, ...
9       935  [105, 946, 871, 896, 975, 967, 979, 982, 983, ...
10     1711  [1433, 1619, 1733, 636, 1736, 1746, 1751, 1594...
11     1893  [1910, 1919, 1574, 1911, 1921, 1922, 1904, 189...
12     1952  [1962, 1931, 619, 1963, 1961, 1969, 1957, 1983...
13     1955  [1975, 1981, 2050, 2029, 2033, 2034, 2035, 204...
14     2025  [2120, 2053, 2041, 2086, 2095, 2168, 2169,

In [497]:
def aa_value(one, two, graph):
    """
        Funcion que devuelve para ese par de nodos, el sumatorio de 1/log(N(z)), siendo N(z) el grado del nodo z para todo z 
        perteneciente al conjunto de nodos en comun de ese par de nodos
    """

    # obtengo un iterador de un solo elemento que tiene en la tercera posicion el valor de AA para el par de nodos
    value = nx.adamic_adar_index(graph, [(one, two)])
    
    value_aa = -1
    for u, v, p in value:
        # print(p)
        # itero el iterador, guardando el valor de adar adamic
        value_aa = p

    return value_aa

In [498]:
def getWeights(row, graph):
    """
        Funcion que calcula la suma de todos los pesos de sus enlaces
    """
    
    # primero obtengo la lista de los vecinos
    neighbors_list = row['neighbors']
    
    # print(neighbors_list)
    
    user = row['user_id']
    suma = 0
    
    for elem in neighbors_list:
        # print(suma)
        # print(graph[user][elem]['weight'])
        suma = suma + aa_value(user, elem, graph)
        
    return suma

In [499]:
def getPonderaciones(row, graph):
    """
        Funcion que calcula la ponderacion para cada usuario vecino del de la fila
        Calculo la ponderacion diviendo el peso del enlace que enlaza cada problema con user_id con la suma 
        total de los pesos
    """
    # primero obtengo la lista de los vecinos
    neighbors_list = row['neighbors']
    
    # obtengo la suma de pesos de los enlaces de esa lista
    total_weight = row['total_weight']
    
    # obtengo el id del usuario al que quiero recomendar
    user = row['user_id']
    
    lista_ponderaciones = list()
    
    for elem in neighbors_list:
        
        # obtengo el peso del enlace
        peso_enlace = aa_value(user, elem, graph)
        
        if total_weight == 0:
            ponderacion = 0
        else:
            # hago la ponderacion
            ponderacion = peso_enlace/total_weight
        
        lista_ponderaciones.append(ponderacion)
        
    return lista_ponderaciones
    

In [500]:
# ahora voy a incluir una nueva columna que tenga las ponderaciones (suma de todos los enlaces) de cada usuario
dataframe_user_recomend['total_weight'] = dataframe_user_recomend.apply (lambda row: getWeights(row, graph), axis=1)

print(dataframe_user_recomend)


    user_id                                          neighbors  total_weight
0        18  [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...    177.602655
1        25  [418, 437, 440, 443, 448, 445, 302, 454, 130, ...   2530.604920
2        60  [67, 64, 84, 89, 91, 54, 95, 82, 112, 120, 124...   2578.506015
3       130  [461, 396, 483, 103, 488, 416, 509, 511, 527, ...   6730.666214
4       414  [176, 25, 418, 437, 440, 443, 448, 445, 302, 4...   6363.230536
5       443  [448, 445, 302, 454, 130, 461, 396, 483, 103, ...   5832.916205
6       448  [445, 302, 454, 130, 461, 396, 483, 103, 488, ...   2158.239760
7       689  [709, 713, 676, 720, 724, 758, 771, 712, 722, ...   4555.691661
8       912  [840, 919, 864, 930, 933, 813, 934, 935, 105, ...    975.122088
9       935  [105, 946, 871, 896, 975, 967, 979, 982, 983, ...   6315.494842
10     1711  [1433, 1619, 1733, 636, 1736, 1746, 1751, 1594...   7346.919332
11     1893  [1910, 1919, 1574, 1911, 1921, 1922, 1904, 189...   1642.953958

In [501]:
# ahora voy a incluir una nueva columna que tenga las ponderaciones (suma de todos los enlaces) de cada usuario
dataframe_user_recomend['score'] = dataframe_user_recomend.apply (lambda row: getPonderaciones(row, graph), axis=1)

# de esta forma en dataframe_user_recomend voy a tener las ponderaciones para cada vecino 
print(dataframe_user_recomend)

    user_id                                          neighbors  total_weight  \
0        18  [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...    177.602655   
1        25  [418, 437, 440, 443, 448, 445, 302, 454, 130, ...   2530.604920   
2        60  [67, 64, 84, 89, 91, 54, 95, 82, 112, 120, 124...   2578.506015   
3       130  [461, 396, 483, 103, 488, 416, 509, 511, 527, ...   6730.666214   
4       414  [176, 25, 418, 437, 440, 443, 448, 445, 302, 4...   6363.230536   
5       443  [448, 445, 302, 454, 130, 461, 396, 483, 103, ...   5832.916205   
6       448  [445, 302, 454, 130, 461, 396, 483, 103, 488, ...   2158.239760   
7       689  [709, 713, 676, 720, 724, 758, 771, 712, 722, ...   4555.691661   
8       912  [840, 919, 864, 930, 933, 813, 934, 935, 105, ...    975.122088   
9       935  [105, 946, 871, 896, 975, 967, 979, 982, 983, ...   6315.494842   
10     1711  [1433, 1619, 1733, 636, 1736, 1746, 1751, 1594...   7346.919332   
11     1893  [1910, 1919, 1574, 1911, 19

In [502]:
# elimino la columna ya que no interesa
del dataframe_user_recomend['total_weight']

print(dataframe_user_recomend)

    user_id                                          neighbors  \
0        18  [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...   
1        25  [418, 437, 440, 443, 448, 445, 302, 454, 130, ...   
2        60  [67, 64, 84, 89, 91, 54, 95, 82, 112, 120, 124...   
3       130  [461, 396, 483, 103, 488, 416, 509, 511, 527, ...   
4       414  [176, 25, 418, 437, 440, 443, 448, 445, 302, 4...   
5       443  [448, 445, 302, 454, 130, 461, 396, 483, 103, ...   
6       448  [445, 302, 454, 130, 461, 396, 483, 103, 488, ...   
7       689  [709, 713, 676, 720, 724, 758, 771, 712, 722, ...   
8       912  [840, 919, 864, 930, 933, 813, 934, 935, 105, ...   
9       935  [105, 946, 871, 896, 975, 967, 979, 982, 983, ...   
10     1711  [1433, 1619, 1733, 636, 1736, 1746, 1751, 1594...   
11     1893  [1910, 1919, 1574, 1911, 1921, 1922, 1904, 189...   
12     1952  [1962, 1931, 619, 1963, 1961, 1969, 1957, 1983...   
13     1955  [1975, 1981, 2050, 2029, 2033, 2034, 2035, 204...   
14     202

In [503]:

# creo un nuevo dataframe que agrupa por el usuario, poniendo en la columna de al lado, el vecino
df_separation_neigh = dataframe_user_recomend.groupby(['user_id']).neighbors.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_separation_neigh.columns = ['user_id', 'neighbors']

print(df_separation_neigh)

       user_id  neighbors
0           18       3373
1           18       3403
2           18       3422
3           18       3427
4           18       3432
5           18       3440
6           18       3452
7           18       3428
8           18       3481
9           18       3485
10          18       3493
11          18       3498
12          18       3496
13          18       3499
14          18       3495
15          18       3501
16          18       3517
17          18       3550
18          18       3497
19          18       2023
20          18       3558
21          18       3583
22          18       3585
23          18       3492
24          18       3582
25          18       3509
26          18       3601
27          18       2888
28          18       3705
29          18       3706
...        ...        ...
37587     3969       3847
37588     3969       3859
37589     3969       3862
37590     3969       2040
37591     3969       3845
37592     3969       3854
37593     39

In [504]:
# creo un nuevo dataframe que agrupa por el usuario, poniendo en la columna de al lado, el score
df_separation_pond = dataframe_user_recomend.groupby(['user_id']).score.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_separation_pond.columns = ['user_id', 'score']

print(df_separation_pond)

       user_id     score
0           18  0.001778
1           18  0.001778
2           18  0.001778
3           18  0.001778
4           18  0.001778
5           18  0.001778
6           18  0.001778
7           18  0.001778
8           18  0.001778
9           18  0.001778
10          18  0.001778
11          18  0.001778
12          18  0.001778
13          18  0.001778
14          18  0.001778
15          18  0.001778
16          18  0.001778
17          18  0.001778
18          18  0.001778
19          18  0.001778
20          18  0.001778
21          18  0.001778
22          18  0.001778
23          18  0.001778
24          18  0.001778
25          18  0.001778
26          18  0.001778
27          18  0.001778
28          18  0.001778
29          18  0.001778
...        ...       ...
37587     3969  0.000799
37588     3969  0.001158
37589     3969  0.001158
37590     3969  0.001746
37591     3969  0.001158
37592     3969  0.001881
37593     3969  0.001634
37594     3969  0.001158


In [505]:
df_separation = df_separation_neigh

df_separation['score'] = df_separation_pond['score']

# creo ahora el nuevo dataframe gracias a los dos df anteriores que eran auxiliares
print(df_separation)

       user_id  neighbors     score
0           18       3373  0.001778
1           18       3403  0.001778
2           18       3422  0.001778
3           18       3427  0.001778
4           18       3432  0.001778
5           18       3440  0.001778
6           18       3452  0.001778
7           18       3428  0.001778
8           18       3481  0.001778
9           18       3485  0.001778
10          18       3493  0.001778
11          18       3498  0.001778
12          18       3496  0.001778
13          18       3499  0.001778
14          18       3495  0.001778
15          18       3501  0.001778
16          18       3517  0.001778
17          18       3550  0.001778
18          18       3497  0.001778
19          18       2023  0.001778
20          18       3558  0.001778
21          18       3583  0.001778
22          18       3585  0.001778
23          18       3492  0.001778
24          18       3582  0.001778
25          18       3509  0.001778
26          18       3601  0

In [506]:
# ahora voy a ordenar en funcion del score de mayor a menor para cada usuario
# ahora lo que quiero es ordenar los vecinos por cada usuario en funcion de su ponderacion
# primero ordeno por su valor de user y luego por el de ponderacion, de forma que quedan ordenador por su valor de ponderacion
df_separation = df_separation.sort_values(by=['user_id', 'score'], ascending=False)
print(df_separation)

       user_id  neighbors     score
37120     3969        103  0.005494
37098     3969         62  0.005494
37334     3969       1711  0.005493
37097     3969        206  0.005493
37406     3969       1949  0.005489
37116     3969        130  0.005488
37158     3969        517  0.005483
37164     3969        613  0.005478
37542     3969       3495  0.005473
37134     3969        518  0.005472
37363     3969       1969  0.005437
37040     3969       3963  0.005420
37376     3969       2050  0.005382
37286     3969        847  0.005270
37355     3969       1922  0.005268
37078     3969        191  0.005267
37105     3969        414  0.005265
37549     3969       3583  0.005259
37531     3969       3427  0.005041
37370     3969       1967  0.005007
37366     3969       1976  0.004913
37210     3969        983  0.004897
37111     3969        443  0.004865
37270     3969       1264  0.004864
37109     3969        437  0.004787
37218     3969        986  0.004787
37055     3969         89  0

In [507]:
# elimino la columna ya que no interesa
del df_separation['score']

print(df_separation)

       user_id  neighbors
37120     3969        103
37098     3969         62
37334     3969       1711
37097     3969        206
37406     3969       1949
37116     3969        130
37158     3969        517
37164     3969        613
37542     3969       3495
37134     3969        518
37363     3969       1969
37040     3969       3963
37376     3969       2050
37286     3969        847
37355     3969       1922
37078     3969        191
37105     3969        414
37549     3969       3583
37531     3969       3427
37370     3969       1967
37366     3969       1976
37210     3969        983
37111     3969        443
37270     3969       1264
37109     3969        437
37218     3969        986
37055     3969         89
37430     3969       2226
37156     3969        610
37159     3969        609
...        ...        ...
575         18       3227
170         18        103
45          18       3720
55          18       3735
87          18       3889
103         18         64
106         

In [508]:
# ahora tengo que hacer un nuevo dataframe con usuario y vecino
# hago primero la agrupacion por usuario
grouped_r = df_separation.groupby('user_id')

# hago la agregacion en una lista 
df_aux = grouped_r.aggregate(lambda x:list(x))

print(df_aux)

                                                 neighbors
user_id                                                   
18       [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...
25       [103, 62, 1711, 206, 847, 437, 1949, 130, 935,...
60       [103, 62, 1711, 206, 847, 437, 1949, 130, 935,...
130      [103, 62, 1711, 206, 847, 1949, 1922, 191, 89,...
414      [103, 62, 1711, 206, 847, 1922, 1949, 130, 191...
443      [103, 62, 1711, 206, 847, 1949, 191, 130, 1922...
448      [103, 62, 206, 437, 935, 2226, 1711, 847, 1949...
689      [103, 62, 1711, 206, 847, 89, 1922, 437, 935, ...
912      [840, 1436, 1485, 1504, 1619, 1751, 1922, 1962...
935      [103, 62, 206, 1711, 437, 847, 1949, 2226, 119...
1711     [103, 62, 206, 847, 437, 1949, 935, 130, 89, 1...
1893     [1962, 2050, 2732, 2802, 3495, 3583, 3492, 220...
1952     [103, 62, 1711, 847, 191, 206, 130, 1922, 517,...
1955     [2050, 191, 414, 689, 840, 1264, 1504, 103, 62...
2025     [103, 62, 1711, 206, 2050, 414, 130, 2097, 203.

In [509]:
# voy a crear un nuevo dataframe con la columna user_id y neighbors (con el anterior no se puede trabajar sin indices)
df_recommend = pd.DataFrame({'user_id':sorted(user_list_to_recommend), 'neighbors':df_aux['neighbors'].tolist()})

print(df_recommend)

                                            neighbors  user_id
0   [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...       18
1   [103, 62, 1711, 206, 847, 437, 1949, 130, 935,...       25
2   [103, 62, 1711, 206, 847, 437, 1949, 130, 935,...       60
3   [103, 62, 1711, 206, 847, 1949, 1922, 191, 89,...      130
4   [103, 62, 1711, 206, 847, 1922, 1949, 130, 191...      414
5   [103, 62, 1711, 206, 847, 1949, 191, 130, 1922...      443
6   [103, 62, 206, 437, 935, 2226, 1711, 847, 1949...      448
7   [103, 62, 1711, 206, 847, 89, 1922, 437, 935, ...      689
8   [840, 1436, 1485, 1504, 1619, 1751, 1922, 1962...      912
9   [103, 62, 206, 1711, 437, 847, 1949, 2226, 119...      935
10  [103, 62, 206, 847, 437, 1949, 935, 130, 89, 1...     1711
11  [1962, 2050, 2732, 2802, 3495, 3583, 3492, 220...     1893
12  [103, 62, 1711, 847, 191, 206, 130, 1922, 517,...     1952
13  [2050, 191, 414, 689, 840, 1264, 1504, 103, 62...     1955
14  [103, 62, 1711, 206, 2050, 414, 130, 2097, 203...  

In [510]:
def getProblemsFromSimilarUSers(row, df_users, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de la concatenacion de listas de problemas que han
        hecho los usuarios similares a ese. Además eliminara los problemas que ya haya hecho el usuario
    """
   
    
    # obtengo la lista de problemas que ha hecho el usuario en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['user_id'] == row['user_id']]
    list_problems_user = list(list_problems_users['list_problem_id'])[0]
    
    
   
    # lista resultante de la concatenacion de las listas de problemas de los usuarios similares
    list_result = list()
    
    # obtengo la longitud de la lista de vecinos de ese usuario
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de usuarios vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de problemas que ha hecho el usuario similar
        list_problems_df = df_users[df_users['user_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_problems_df['list_problem_id'])[0]
        
        # aqui hago el filtro para que no se incluyan los problemas que ya ha hecho el usuario
        list_problems = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora concateno el resultado
        list_result = list_result + list_problems
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [511]:
# ahora para cada lista de de usuarios, hacer una lista de los problemas realizados por esos usuarios, 
# que no los haya realizado ya el usuario
df_recommend['list_problems'] = df_recommend.apply (lambda row: getProblemsFromSimilarUSers(row, df_users, df_users_recommend), axis=1)

print(df_recommend)


                                            neighbors  user_id  \
0   [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...       18   
1   [103, 62, 1711, 206, 847, 437, 1949, 130, 935,...       25   
2   [103, 62, 1711, 206, 847, 437, 1949, 130, 935,...       60   
3   [103, 62, 1711, 206, 847, 1949, 1922, 191, 89,...      130   
4   [103, 62, 1711, 206, 847, 1922, 1949, 130, 191...      414   
5   [103, 62, 1711, 206, 847, 1949, 191, 130, 1922...      443   
6   [103, 62, 206, 437, 935, 2226, 1711, 847, 1949...      448   
7   [103, 62, 1711, 206, 847, 89, 1922, 437, 935, ...      689   
8   [840, 1436, 1485, 1504, 1619, 1751, 1922, 1962...      912   
9   [103, 62, 206, 1711, 437, 847, 1949, 2226, 119...      935   
10  [103, 62, 206, 847, 437, 1949, 935, 130, 89, 1...     1711   
11  [1962, 2050, 2732, 2802, 3495, 3583, 3492, 220...     1893   
12  [103, 62, 1711, 847, 191, 206, 130, 1922, 517,...     1952   
13  [2050, 191, 414, 689, 840, 1264, 1504, 103, 62...     1955   
14  [103, 

In [512]:
def delRepetitions(row):
    """
        Funcion auxiliar para evitar que salgan repeticiones en las recomendaciones. Saco la lista de posibles 
        recomendaciones con valores unicos
    """
    conjunto_vacio = set()
    
    # esto sirve para que se haga mas rapido la comprobacion de si el elemento esta en la lista o no
    function_add = conjunto_vacio.add
    
    # hago la lista intensional, para mantener el orden dado en la lista original
    return [x for x in row['list_problems'] if not (x in conjunto_vacio or function_add(x))]

In [513]:
# ahora tengo la lista de posibles problemas a recomendar para cada usuario

# voy a sacar primero una lista sin repeticiones
# ahora voy a crear una nueva columna que contenga la lista de problemas sin repeticiones
df_recommend['lista_problemas_unique'] = df_recommend.apply(lambda row: delRepetitions(row), axis=1)

print(df_recommend)

                                            neighbors  user_id  \
0   [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...       18   
1   [103, 62, 1711, 206, 847, 437, 1949, 130, 935,...       25   
2   [103, 62, 1711, 206, 847, 437, 1949, 130, 935,...       60   
3   [103, 62, 1711, 206, 847, 1949, 1922, 191, 89,...      130   
4   [103, 62, 1711, 206, 847, 1922, 1949, 130, 191...      414   
5   [103, 62, 1711, 206, 847, 1949, 191, 130, 1922...      443   
6   [103, 62, 206, 437, 935, 2226, 1711, 847, 1949...      448   
7   [103, 62, 1711, 206, 847, 89, 1922, 437, 935, ...      689   
8   [840, 1436, 1485, 1504, 1619, 1751, 1922, 1962...      912   
9   [103, 62, 206, 1711, 437, 847, 1949, 2226, 119...      935   
10  [103, 62, 206, 847, 437, 1949, 935, 130, 89, 1...     1711   
11  [1962, 2050, 2732, 2802, 3495, 3583, 3492, 220...     1893   
12  [103, 62, 1711, 847, 191, 206, 130, 1922, 517,...     1952   
13  [2050, 191, 414, 689, 840, 1264, 1504, 103, 62...     1955   
14  [103, 

In [514]:
def getListProblemsFromSimilarUSers(row, df_users, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de listas la concatenacion de listas de problemas que han
        hecho los usuarios similares a ese. Además eliminara los problemas que ya haya hecho el usuario
    """
    
    # obtengo la lista de problemas que ha hecho el usuario en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['user_id'] == row['user_id']]
    list_problems_user = list(list_problems_users['list_problem_id'])[0]
   
    # lista resultante 
    list_result = list(list())
    
    # obtengo la longitud de la lista de vecinos de ese usuario
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de usuarios vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de problemas que ha hecho el usuario similar
        list_problems_df = df_users[df_users['user_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_problems_df['list_problem_id'])[0]
        
        # aqui hago el filtro para que no se incluyan los problemas que ya ha hecho el usuario
        list_problems = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora incluyo la lista en la lista
        list_result.append(list_problems)
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [515]:
# ahora, para la lista de problemas que se pueden recomendar, tengo que hacer la suma de las ponderaciones
# voy a crear una columna en la cual se guarde una lista de listas de problemas, en las que cada posicion coincidira con el 
# usuario que las haya hecho, sin que se guarden los problemas que ha hecho el usuario al que quiero recomendar
df_recommend['lista_problemas_por_user'] = df_recommend.apply(lambda row: getListProblemsFromSimilarUSers(row, df_users, df_users_recommend), axis=1)

print(df_recommend)

                                            neighbors  user_id  \
0   [3373, 3403, 3422, 3427, 3432, 3440, 3452, 342...       18   
1   [103, 62, 1711, 206, 847, 437, 1949, 130, 935,...       25   
2   [103, 62, 1711, 206, 847, 437, 1949, 130, 935,...       60   
3   [103, 62, 1711, 206, 847, 1949, 1922, 191, 89,...      130   
4   [103, 62, 1711, 206, 847, 1922, 1949, 130, 191...      414   
5   [103, 62, 1711, 206, 847, 1949, 191, 130, 1922...      443   
6   [103, 62, 206, 437, 935, 2226, 1711, 847, 1949...      448   
7   [103, 62, 1711, 206, 847, 89, 1922, 437, 935, ...      689   
8   [840, 1436, 1485, 1504, 1619, 1751, 1922, 1962...      912   
9   [103, 62, 206, 1711, 437, 847, 1949, 2226, 119...      935   
10  [103, 62, 206, 847, 437, 1949, 935, 130, 89, 1...     1711   
11  [1962, 2050, 2732, 2802, 3495, 3583, 3492, 220...     1893   
12  [103, 62, 1711, 847, 191, 206, 130, 1922, 517,...     1952   
13  [2050, 191, 414, 689, 840, 1264, 1504, 103, 62...     1955   
14  [103, 

In [516]:
# ahora voy a separar cada user-problema_a_recomendar para hacer la cuenta
# creo un nuevo dataframe que agrupa por el primer problema y tiene su posible recomendacion
new_df_separation = df_recommend.groupby(['user_id']).lista_problemas_unique.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

new_df_separation.columns = ['user_id', 'recommendation']

print(new_df_separation)

      user_id  recommendation
0          18              62
1          18             502
2          18             509
3          18             105
4          18             307
5          18             166
6          18              39
7          18               2
8          18               6
9          18               8
10         18              10
11         18              13
12         18              33
13         18             139
14         18              35
15         18              44
16         18              49
17         18             150
18         18             183
19         18             327
20         18             191
21         18             325
22         18             390
23         18             254
24         18             134
25         18             162
26         18             109
27         18             253
28         18              78
29         18             171
...       ...             ...
9923     3969             231
9924     3

In [517]:
def getScoring(row, df_recommend):
    """
        Funcion en la que por cada usuario-problema, hace el scoring para ese problema (para ese usuario)
        sumando todas las ponderaciones de ese problema para ese usuario (ponderacion: 1/(pos+1))
    """
    # obtengo la lista de listas de problemas que han hecho los vecinos del usuario
    # los vecinos estan ordenados por su ponderacion    
    list_problems_users_df = df_recommend[df_recommend['user_id'] == row['user_id']]
    list_problems_per_user = list(list_problems_users_df['lista_problemas_por_user'])[0]
    
    
    # obtengo el problema del que quiero calcular el scoring
    problem = row['recommendation']
    # print(problem)
    
    suma = 0
    k = len(list_problems_per_user)
    
    # por cada lista de la lista, saco la ponderacion de su correspondiente usuario (el usuario que lo ha hecho)
    # si problem aparece en la lista, sumo esa ponderacion
    for i in range(0, k):
        lista_a_comprobar = list_problems_per_user[i]
        # print(lista_a_comprobar)
        if problem in lista_a_comprobar:
            # se suma la nueva ponderacion 
            suma = suma + (1/(i+1))
    
    return suma

In [518]:
# ahora voy a calcular el scoring para cada problema
new_df_separation['score'] = new_df_separation.apply(lambda row: getScoring(row, df_recommend), axis=1)

print(new_df_separation)

      user_id  recommendation     score
0          18              62  1.326480
1          18             502  1.222949
2          18             509  1.065290
3          18             105  1.219128
4          18             307  1.238533
5          18             166  1.041055
6          18              39  3.686971
7          18               2  1.413869
8          18               6  0.897186
9          18               8  0.632846
10         18              10  0.838282
11         18              13  1.140292
12         18              33  2.361286
13         18             139  0.690328
14         18              35  0.705802
15         18              44  2.923732
16         18              49  1.628849
17         18             150  2.618633
18         18             183  1.942558
19         18             327  2.710708
20         18             191  1.404570
21         18             325  2.109117
22         18             390  1.020155
23         18             254  1.473282


In [519]:
# ahora voy a ordenar en funcion del score de mayor a menor para cada usuario
# ahora lo que quiero es ordenar los problemas por cada usuario en funcion de su ponderacion
# primero ordeno por su valor de user y luego por el de ponderacion, de forma que quedan ordenador por su valor de ponderacion
new_df_separation = new_df_separation.sort_values(by=['user_id', 'score'], ascending=False)
print(new_df_separation)

      user_id  recommendation     score
9864     3969             327  4.607601
9835     3969              44  4.582991
9790     3969              33  4.574699
9852     3969             109  4.364036
9913     3969             162  4.308819
9912     3969             159  4.288995
9796     3969             187  4.275817
9908     3969             141  4.100695
9918     3969             209  4.095451
9906     3969             136  4.083418
9855     3969             203  4.028106
9814     3969               2  3.993035
9798     3969             254  3.878221
9911     3969             155  3.853766
9793     3969              70  3.812408
9925     3969             237  3.780521
9805     3969              49  3.779765
9838     3969              13  3.727587
9907     3969             139  3.720352
9841     3969             390  3.563076
9858     3969             258  3.482535
9791     3969              51  3.473222
9849     3969              62  3.442355
9866     3969             310  3.424664


In [520]:
# hago primero la agrupacion por usuario
grouped_r = new_df_separation.groupby('user_id')

# hago la agregacion en una lista 
df_recommend_final = grouped_r.aggregate(lambda x:list(x))

print(df_recommend_final)

                                            recommendation  \
user_id                                                      
18       [39, 44, 327, 150, 33, 325, 183, 49, 134, 162,...   
25       [39, 109, 2, 183, 254, 33, 44, 327, 134, 159, ...   
60       [39, 109, 183, 44, 33, 187, 141, 162, 258, 209...   
130      [254, 49, 258, 253, 256, 436, 259, 510, 307, 1...   
414      [39, 159, 390, 253, 441, 70, 100, 310, 256, 47...   
443      [33, 187, 134, 162, 49, 136, 155, 203, 13, 191...   
448      [39, 109, 2, 254, 183, 187, 258, 327, 141, 159...   
689      [327, 254, 187, 258, 49, 150, 325, 139, 203, 1...   
912      [39, 187, 33, 44, 254, 136, 258, 203, 150, 256...   
935      [39, 183, 44, 150, 13, 237, 4, 145, 336, 245, ...   
1711     [259, 309, 251, 437, 308, 352, 53, 374, 231, 4...   
1893     [44, 33, 183, 109, 327, 209, 258, 162, 325, 13...   
1952     [39, 150, 109, 254, 187, 141, 209, 155, 2, 134...   
1955     [39, 44, 33, 183, 2, 162, 150, 187, 141, 254, ...   
2025    

In [521]:
def getKrecomFinal(row, k):
    """
        Funcion que saca las k mejores recomendaciones para el usuario
        Lo que hace es coger los primeros k valores de la lista de recomendaciones
    """
    if k == 1:
        value = list()
        value.append(row['recommendation'][:k])
        return value
    else:
        return row['recommendation'][:k]

In [522]:
k = 10
# ahora saco los k mejores problemas para cada usuario
df_recommend_final['k_recommendation'] = df_recommend_final.apply(lambda row: getKrecomFinal(row, k), axis=1)

print(df_recommend_final)

                                            recommendation  \
user_id                                                      
18       [39, 44, 327, 150, 33, 325, 183, 49, 134, 162,...   
25       [39, 109, 2, 183, 254, 33, 44, 327, 134, 159, ...   
60       [39, 109, 183, 44, 33, 187, 141, 162, 258, 209...   
130      [254, 49, 258, 253, 256, 436, 259, 510, 307, 1...   
414      [39, 159, 390, 253, 441, 70, 100, 310, 256, 47...   
443      [33, 187, 134, 162, 49, 136, 155, 203, 13, 191...   
448      [39, 109, 2, 254, 183, 187, 258, 327, 141, 159...   
689      [327, 254, 187, 258, 49, 150, 325, 139, 203, 1...   
912      [39, 187, 33, 44, 254, 136, 258, 203, 150, 256...   
935      [39, 183, 44, 150, 13, 237, 4, 145, 336, 245, ...   
1711     [259, 309, 251, 437, 308, 352, 53, 374, 231, 4...   
1893     [44, 33, 183, 109, 327, 209, 258, 162, 325, 13...   
1952     [39, 150, 109, 254, 187, 141, 209, 155, 2, 134...   
1955     [39, 44, 33, 183, 2, 162, 150, 187, 141, 254, ...   
2025    

In [523]:
# elimino las columnas que no me interesan
del df_recommend_final['recommendation']
del df_recommend_final['score']

print(df_recommend_final)

                                          k_recommendation
user_id                                                   
18          [39, 44, 327, 150, 33, 325, 183, 49, 134, 162]
25           [39, 109, 2, 183, 254, 33, 44, 327, 134, 159]
60         [39, 109, 183, 44, 33, 187, 141, 162, 258, 209]
130      [254, 49, 258, 253, 256, 436, 259, 510, 307, 174]
414       [39, 159, 390, 253, 441, 70, 100, 310, 256, 471]
443        [33, 187, 134, 162, 49, 136, 155, 203, 13, 191]
448        [39, 109, 2, 254, 183, 187, 258, 327, 141, 159]
689      [327, 254, 187, 258, 49, 150, 325, 139, 203, 191]
912        [39, 187, 33, 44, 254, 136, 258, 203, 150, 256]
935          [39, 183, 44, 150, 13, 237, 4, 145, 336, 245]
1711     [259, 309, 251, 437, 308, 352, 53, 374, 231, 486]
1893      [44, 33, 183, 109, 327, 209, 258, 162, 325, 136]
1952       [39, 150, 109, 254, 187, 141, 209, 155, 2, 134]
1955         [39, 44, 33, 183, 2, 162, 150, 187, 141, 254]
2025        [39, 33, 109, 209, 187, 141, 136, 2, 134, 20

In [524]:
# ahora tengo que filtrar df_users_eval para que solo contenga las filas de los usuarios a los que hay que recomendar

df_users_eval_filter = df_users_eval[df_users_eval['user_id'].isin(user_list_to_recommend)]
print(df_users_eval_filter)

                                       list_problem_id  user_id
4               [241, 128, 124, 256, 404, 469, 70, 86]       18
5                    [33, 29, 254, 308, 445, 253, 166]       25
8    [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...       60
13                       [254, 253, 308, 166, 49, 445]      130
22   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...      414
26             [134, 187, 256, 251, 90, 174, 147, 438]      443
27   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...      448
34                     [327, 23, 150, 349, 336, 6, 49]      689
44   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...      912
46          [95, 383, 262, 122, 44, 81, 252, 145, 336]      935
70   [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...     1711
73                       [471, 469, 355, 95, 243, 436]     1893
78         [374, 195, 109, 436, 95, 10, 446, 340, 379]     1952
80                            [44, 213, 136, 162, 243]     1955
90             [136, 253, 187, 342, 203,

In [525]:
list_eval_problems = df_users_eval_filter['list_problem_id'].tolist()
list_recom_problems = df_recommend_final['k_recommendation'].tolist()


# meto toda la informacion en un dataframe para obtener las metricas
set_df_metric = {'user_id': user_list_to_recommend, 'eval_problems': list_eval_problems, 'recom_problems': list_recom_problems}
metric_df = pd.DataFrame.from_dict(set_df_metric)

print(metric_df)

                                        eval_problems  \
0              [241, 128, 124, 256, 404, 469, 70, 86]   
1                   [33, 29, 254, 308, 445, 253, 166]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...   
3                       [254, 253, 308, 166, 49, 445]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...   
5             [134, 187, 256, 251, 90, 174, 147, 438]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...   
7                     [327, 23, 150, 349, 336, 6, 49]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...   
11                      [471, 469, 355, 95, 243, 436]   
12        [374, 195, 109, 436, 95, 10, 446, 340, 379]   
13                           [44, 213, 136, 162, 243]   
14            [136, 253, 187, 342, 203, 33, 441, 508]   
15  [136, 253, 187, 33, 441, 342, 209, 162, 155, 4...   
16                           [1

In [526]:
def one_hit(row):
    """
        Funcion que implementa la metrica one hit. Devuelve un 1 si para un usuarios dado, al menos uno
        de los problemas que se le ha recomendado ha sido realizado por ese usuario en el evaluation_set. 
        Cero si no hay ningun problema de los recomendados que haya sido realizado por el usuario
    """
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    if len(num_problems_common) >= 1:
        return 1
    else:
        return 0

In [527]:
def mrr(row): 
    """
        Funcion que va a implementar la metrica de evaluacion mrr:
        mrr = 1/ranki, donde ranki es la posicion del primer item correcto
    """

    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    if len(num_problems_common) >= 1:

        # hago la busqueda del primer elemento que esta en la lista de recomendados
        fst_correct_item = -1
        encontrado = False
        i = 0
        while (i < len(row['recom_problems'])) and (encontrado == False):
            if row['recom_problems'][i] in row['eval_problems']:
                # fst_correct_item = row['recom_problems'][i]
                # print(fst_correct_item)
                ranki = i + 1
                encontrado = True
            else:
                i = i + 1
                
        return (1/ranki)

    else:
        return 0

In [528]:
def precision(row):
    """
        Funcion que va a implementar la metrica precision en k: 
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los recomendados
    """
    
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['recom_problems']))

In [529]:
def recall(row):
    """
        Funcion que implementa la metrica recall
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los evaluados
    """
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['eval_problems']))

In [530]:
def f1(row):
    """
        Funcion que calcula el f1 en funcion de precision y recall
    """
    denominador = row['precision'] + row['recall']
    
    if denominador == 0:
        return 0
    else:
        return (2 * row['precision'] * row['recall']) / denominador

In [531]:
# ahora voy a calcular una metrica para cada usuario
metric_df['one_hit'] = metric_df.apply(lambda row: one_hit(row), axis=1)
metric_df['mrr'] = metric_df.apply(lambda row: mrr(row), axis=1)
metric_df['precision'] = metric_df.apply(lambda row: precision(row), axis=1)
metric_df['recall'] = metric_df.apply(lambda row: recall(row), axis=1)
metric_df['f1'] = metric_df.apply(lambda row: f1(row), axis=1)
print(metric_df)

                                        eval_problems  \
0              [241, 128, 124, 256, 404, 469, 70, 86]   
1                   [33, 29, 254, 308, 445, 253, 166]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...   
3                       [254, 253, 308, 166, 49, 445]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...   
5             [134, 187, 256, 251, 90, 174, 147, 438]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...   
7                     [327, 23, 150, 349, 336, 6, 49]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...   
11                      [471, 469, 355, 95, 243, 436]   
12        [374, 195, 109, 436, 95, 10, 446, 340, 379]   
13                           [44, 213, 136, 162, 243]   
14            [136, 253, 187, 342, 203, 33, 441, 508]   
15  [136, 253, 187, 33, 441, 342, 209, 162, 155, 4...   
16                           [1

In [532]:
# calculo la media de las metricas

result_one_hit = metric_df['one_hit'].mean()
result_precision = metric_df['precision'].mean()
result_mrr = metric_df['mrr'].mean()
result_recall = metric_df['recall'].mean()
result_f1 = metric_df['f1'].mean()

print("One hit ----------")
print(result_one_hit)
print("Precision ----------")
print(result_precision)
print("Mrr  ----------")
print(result_mrr)
print("Recall  ----------")
print(result_recall)
print("F1  ----------")
print(result_f1)

One hit ----------
0.9076923076923077
Precision ----------
0.2815384615384615
Mrr  ----------
0.5707875457875456
Recall  ----------
0.29484228661496653
F1  ----------
0.2721132227298291


In [533]:

f = open("C:/hlocal/TFM/vot_ord_pond", 'a')
f.write(str(result_one_hit) + '\t' + str(result_precision) + '\t' + str(result_mrr) + '\t' + str(result_recall) + '\t' +  str(result_f1) + '\n') 
f.close()